# 3. *Code*

## 3.1 Импорт библиотек

In [1]:
import numpy as np
from nltk.tokenize import WordPunctTokenizer
import gensim.downloader as api
# and some other your dependencies
from gensim.models.phrases import Phrases, Phraser
import spacy  # For preprocessing
import nltk
import re
from scipy.spatial.distance import cosine
from tqdm import tqdm
from nltk import word_tokenize, PorterStemmer
import string
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import xgboost as xgb
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## 3.2 Загрузка данных

In [2]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

data = list(open("./quora.txt", encoding="utf-8"))

--2024-09-22 17:56:34--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/p0t2dw6oqs6oxpd6zz534/quora.txt?rlkey=bjupppwua4zmd4elz8octecy9&dl=1 [following]
--2024-09-22 17:56:34--  https://www.dropbox.com/scl/fi/p0t2dw6oqs6oxpd6zz534/quora.txt?rlkey=bjupppwua4zmd4elz8octecy9&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc900ac9904c0ede68ce3e711986.dl.dropboxusercontent.com/cd/0/inline/CbG86bWTGuTjwYPSoft_u9rwGf3KxndidKFMlGzAx-jSRcNDdnhGvTJbEazfzRzWlIWY0Llt0wcEDr3Y59cL6d_Kl6yDbcWMX42JlpfbBU-jYkg3uEpRCjCdAEOBfJ1a4r4/file?dl=1# [following]
--2024-09-22 17:56:35--  https://uc900ac9904c0ede68ce3e711986.dl.dropboxusercontent.com/cd/0/inline/CbG86bWTGuT

### 3.1.1 Предобработка данных

In [3]:
def preprocess_phrase(phrase):
    """Preprocesses a phrase by converting to lowercase, removing punctuation,
    tokenizing, and stemming.
    """
    # Convert to lowercase and remove punctuation
    phrase = phrase.lower().translate(str.maketrans('', '', string.punctuation))

    # Tokenize and stem
    tokens = word_tokenize(phrase)
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]

    return stemmed_tokens

In [4]:
filteted_data = [preprocess_phrase(row) for row in data]

## 3.3. Модель

### 3.3.1 Эмбединги


In [5]:
#Fit TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit([' '.join(phrase) for phrase in filteted_data])

def get_phrase_embedding(phrase, preprose=False):
    """
    Convert phrase to a vector by aggregating it's word embeddings.
    """
    if preprose:
      phrase = preprocess_phrase(phrase)
    vector = np.zeros([model.vector_size], dtype='float32')
    word_vectors = []
    tfidf_values = []

    # Get TF-IDF values for the phrase, handling potential KeyError
    tfidf = vectorizer.transform([' '.join(phrase)])

    for word in phrase:
      if word in model.key_to_index and word in vectorizer.vocabulary_:
        word_vectors.append(model.get_vector(word))
        tfidf_values.append(tfidf[0, vectorizer.vocabulary_[word]])

    if word_vectors:
      word_vectors = np.array(word_vectors)
      tfidf_values = np.array(tfidf_values)
      vector = np.average(word_vectors, axis=0, weights=tfidf_values)

    return vector

In [6]:
tokenizer = WordPunctTokenizer()
model = api.load('glove-twitter-100')

In [7]:
# compute vector embedding for all lines in data
data_vectors = np.array([get_phrase_embedding(row) for row in filteted_data])

### 3.3.2 Xgbranker




In [8]:
def is_exact_match(query, candidate):
    # Ensure both inputs are strings
    query_str = str(query)
    candidate_str = str(candidate)

    # Remove punctuation and convert to lowercase
    query_clean = query_str.translate(str.maketrans('', '', string.punctuation)).lower()
    candidate_clean = candidate_str.translate(str.maketrans('', '', string.punctuation)).lower()

    # Return True if the cleaned strings are exactly the same
    return query_clean == candidate_clean

def simple_text_features(query, candidate):
    # Ensure both inputs are strings
    query_str = str(query)
    candidate_str = str(candidate)

    # Remove punctuation
    query_clean = query_str.translate(str.maketrans('', '', string.punctuation)).lower()
    candidate_clean = candidate_str.translate(str.maketrans('', '', string.punctuation)).lower()

    # Tokenize sentences into words
    query_words = set(query_clean.split())
    candidate_words = set(candidate_clean.split())

    # Common characters
    common_chars = len(set(query_clean) & set(candidate_clean))

    # Common words
    common_words = len(query_words & candidate_words)

    # Length of each sentence (number of characters)
    len_query = len(query_clean)
    len_candidate = len(candidate_clean)

    # Length of each sentence (number of words)
    len_query_words = len(query_words)
    len_candidate_words = len(candidate_words)

    # Absolute difference in length of the sentences (character length)
    length_diff = abs(len_query - len_candidate)

    # Character overlap ratio
    char_overlap_ratio = common_chars / min(len_query, len_candidate) if min(len_query, len_candidate) > 0 else 0

    # Word overlap ratio
    word_overlap_ratio = common_words / min(len_query_words, len_candidate_words) if min(len_query_words, len_candidate_words) > 0 else 0

    # Jaccard similarity for word sets
    jaccard_sim = common_words / len(query_words | candidate_words) if len(query_words | candidate_words) > 0 else 0

    # Compile features into a dictionary
    features = {
        'common_chars': common_chars,
        'common_words': common_words,
        'len_query': len_query,
        'len_candidate': len_candidate,
        'length_diff': length_diff,
        'char_overlap_ratio': char_overlap_ratio,
        'word_overlap_ratio': word_overlap_ratio,
        'jaccard_similarity': jaccard_sim
    }

    return features


def create_ranking_dataset(queries, candidate_questions):
    data = []

    # Iterate over queries
    for i, query in enumerate(queries):
        candidates = candidate_questions[i]

        for j, candidate in enumerate(candidates):
            # Check if exact match
            if is_exact_match(query, candidate):
                target = 1
            else:
                target = 0

            # Assuming you have embeddings for both questions
            query_embedding = data_vectors[i]
            candidate_embedding = data_vectors[j]

            # Example: Add cosine similarity as a feature
            cosine_sim = cosine(query_embedding, candidate_embedding)
            txt_features = simple_text_features(query, candidate)
            feats = {
                'query': query,
                'candidate': candidate,
                'cosine_sim': cosine_sim,
                'target': target
            }
            feats.update(txt_features)
            data.append(feats)

    return pd.DataFrame(data)

def rank_candidates(model, query, candidates):
    data = []

    for candidate in candidates:
        query_embedding = get_phrase_embedding(query)
        candidate_embedding = get_phrase_embedding(candidate)
        cosine_sim = cosine(query_embedding, candidate_embedding)
        txt_features = simple_text_features(query, candidate)
        feats = {'cosine_sim': cosine_sim}
        feats.update(txt_features)
        data.append(feats)

    # Convert to DataFrame
    df = pd.DataFrame(data)

    # Predict ranking scores
    X = df.values
    dtest = xgb.DMatrix(X)
    scores = model.predict(dtest)

    # Sort candidates by scores (higher is better)
    sorted_candidates = [c for _, c in sorted(zip(scores, candidates), reverse=True)]

    return sorted_candidates


def prepare_data_for_xgb(df):
    # Features and target
    X = df.drop(['query', 'candidate', 'target'], axis=1).values
    y = df['target'].values

    # Grouping by query for ranking
    group_sizes = df.groupby('query').size().values

    return X, y, group_sizes

In [9]:
df_data = pd.DataFrame(data)

df = create_ranking_dataset(queries=df_data,
                            candidate_questions=df_data)
X, y, group_sizes = prepare_data_for_xgb(df)

dtrain = xgb.DMatrix(X, label=y)
dtrain.set_group(group_sizes)

params = {
    'objective': 'rank:pairwise',  # Ranking objective
    'learning_rate': 0.1,
    'max_depth': 4,
    'eval_metric': 'ndcg',
}

# Train the model
model_xgb = xgb.train(params, dtrain, num_boost_round=100)

In [12]:
def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    emb = get_phrase_embedding(query)
    similarities = np.dot(data_vectors, emb) / (np.linalg.norm(data_vectors, axis=1) * np.linalg.norm(emb))


    indices = np.argpartition(-similarities, k)[:k]

    return [data[ind] for ind in indices]

## 3.4. Применение модели

In [13]:
search = preprocess_phrase("Who is trump?")
top = 10
pre_top = find_nearest(query=search, k=top*5)
result = pre_top[0:top//2]
xgb_output = rank_candidates(model_xgb, search, pre_top[len(result):])
result + xgb_output[:(top-len(result))]

['Based on his personality what country would you say Donald Trump was from if Donald Trump were not American?\n',
 'Who would be better for India, Donald Trump or Hillary Clinton?\n',
 "What will be Donald Trump's legacy?\n",
 'What does Donald Trump think of India?\n',
 'What India thinks about Donald Trump?\n',
 "Why did Hillary Clinton go to Donald Trump's wedding in 2005, if Trump is a longtime racist and sexist?\n",
 'Who would be a better president: Hillary Clinton or Donald Trump?\n',
 'Who will win, Donald Trump or Hillary Clinton?\n',
 'Who will win Hillary Clinton Or Donald Trump?\n',
 'Who will own the Trump brand after Trump takes office?\n']